**Author:** Cristiano Landi

# TS Feature Extraction

In [1]:
import pandas as pd
import librosa, os, glob
from tqdm import notebook as tq
from pylab import rcParams
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor

In [2]:
path = 'C:/Users/cri98/Downloads/fma_large/' #To remove files from folders run this "find . -name "*.mp3" -exec mv {} . \;"
entries = os.listdir(path)

In [3]:
from pydub import AudioSegment
from tslearn.piecewise import PiecewiseAggregateApproximation
from tslearn.piecewise import SymbolicAggregateApproximation
from tslearn.piecewise import OneD_SymbolicAggregateApproximation
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)

def estraiListaDataframe(entries=list(), start=0, stop=-1, n_seg=1292, sr=22050, hop_length=512, frame_length=1024):   
    #[start incluso, stop)
    #n_seg=1292 --> = (duration*sr)/hop_length
    
    f = open("features/"+str(start)+"-"+str(stop)+".csv", "wt", 1024*1024*256)
    f.write("id,duration")
    
    for i in range(n_seg):
        f.write(","+str(i))
    f.write("\n")
    
    for entry in tq.tqdm(entries[start:stop]):
        strings = []
        #try:
        sound = AudioSegment.from_mp3(path+entry)
        entryWAV = entry.replace(".mp3", ".wav")
        sound.export(entryWAV, format="wav", parameters=["-ar", str(sr), "-ac", "1"])

        y, sr = librosa.load(entryWAV, sr=sr, mono=True)

        os.remove(entryWAV)
        
        

        #ts_ = scaler.fit_transform(y.reshape(1,-1))

        
        
        #https://librosa.org/doc/main/feature.html#
        rms = librosa.feature.rms(y, frame_length=frame_length, hop_length=hop_length)[0]

        
        

        duration = librosa.get_duration(y=y, sr=sr)
        id_ = entry.replace('.mp3','')
        strings.append("%s,%f" % (id_, duration))

        for i in range(n_seg):
            strings.append(",")
            if i >= len(rms):
                strings.append("0.0")
            else: 
                strings.append(str(rms[i]))

        strings.append("\n")

        f.write(''.join(strings))
        #except:
            #pass
        
    
    f.close()

In [4]:
executor = ThreadPoolExecutor(8) #max_workers

n_split = 8; #number of csv files

intervalli = range(0, len(entries), int(len(entries)/n_split))

df_list = list()
ts = dict()
future = {}
for i in range(len(intervalli) -1):
    future[i] = executor.submit(estraiListaDataframe, entries, intervalli[i], intervalli[i+1])

future[i+1] = executor.submit(estraiListaDataframe, entries, intervalli[i+1], len(entries))

executor.shutdown(wait=True)

# Union of all CSVs

In [5]:
lst = glob.glob("features/*.csv")
#print(lst)

filename='features/complessivo_rms.csv'

out = open(filename, 'w')

f = open(lst[0], 'r')
lines = f.readlines()
out.write(lines[0]+"\n")
f.close()
 
for file in tq.tqdm(lst):
    f = open(file, 'r')
    out.writelines(f.readlines()[1:])
    f.close()
    
out.close()

# Let's try to open the file

In [6]:
df_letto = pd.read_csv("features/complessivo_rms.csv", index_col="id")
df_letto

,duration,0,1,2,3,4,5,6,7,8,...,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291
id,,,,,,,,,,,,,,,,,,,,,
2,29.976599,0.005965,0.030530,0.038409,0.052510,0.081335,0.096664,0.092021,0.083451,0.083296,...,0.167174,0.126127,0.111858,0.095006,0.099327,0.097803,0.107587,0.152822,0.177150,0.000000
3,30.002721,0.009710,0.088201,0.120011,0.128533,0.150989,0.169258,0.205793,0.261233,0.279276,...,0.126499,0.103162,0.147984,0.207548,0.256682,0.254089,0.194205,0.146202,0.119663,0.121850
5,30.002721,0.004292,0.036312,0.051975,0.046670,0.203390,0.283229,0.314644,0.322023,0.293859,...,0.318426,0.312173,0.305558,0.281919,0.234308,0.178978,0.163058,0.143980,0.120217,0.107188
10,29.976599,0.110271,0.142744,0.156790,0.131847,0.125279,0.129176,0.128395,0.171358,0.298087,...,0.296138,0.217364,0.184314,0.176318,0.161866,0.158079,0.158619,0.148111,0.146005,0.000000
20,29.976599,0.245351,0.239747,0.211342,0.181047,0.184478,0.177384,0.142504,0.206317,0.242042,...,0.333991,0.285059,0.176683,0.181024,0.188473,0.168699,0.164692,0.181571,0.183699,0.000000
26,29.976599,0.050055,0.058988,0.062978,0.055367,0.048173,0.042086,0.042418,0.053472,0.049972,...,0.045191,0.043593,0.039672,0.043070,0.053928,0.056892,0.155418,0.288843,0.282220,0.000000
30,29.976599,0.228823,0.253243,0.225191,0.165272,0.161443,0.156461,0.158840,0.157915,0.157949,...,0.242546,0.217658,0.205667,0.201171,0.197343,0.179616,0.168552,0.167429,0.160340,0.000000
46,29.976599,0.143595,0.168848,0.185043,0.173385,0.162525,0.156003,0.152523,0.157803,0.166174,...,0.085848,0.089071,0.111215,0.126269,0.130263,0.124001,0.109992,0.092028,0.081965,0.000000
48,29.976599,0.107023,0.189021,0.234189,0.212959,0.213999,0.214210,0.252316,0.295341,0.285041,...,0.196965,0.200514,0.189053,0.170277,0.176575,0.181547,0.178754,0.161724,0.158578,0.000000
